By ZincCat

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
p = 20
m = 25
n = 20
# np.random.seed(100)
# A = np.random.uniform(0, 1, (n, m, p))
A = np.ones((n, m, p))
for i in range(n):
    A[i] *= i+1
b = np.random.normal(0, 1, m)
# b = np.ones(m)
x0 = np.random.normal(0, 1, (n, p))
# x0 = np.ones((n, p))

In [ ]:
def f(x):
    return np.sum(np.linalg.norm(x, 1, axis=1))
def cons(x, A, b):
    return np.einsum('ijk, ik -> j', A, x) - b
def shrink(x, eps):
    x[abs(x) <= eps] = 0
    x[x > eps] -= eps
    x[x < -eps] += eps
    return x
def update_x(x, A, b, beta, eta, lam):
    for i in range(n):
        x[i] = shrink(x[i] - A[i].T@(lam + beta*cons(x, A, b))/eta[i]/beta, 1/beta/eta[i])
    return x

In [ ]:
eps1 = 1e-5
eps2 = 1e-6
maxBeta = 1000
beta = 1
eta = [(np.linalg.norm(i, 'fro')**2) for i in A]
rho0 = 1.5
maxIter = 1000000

x = x0.copy()
lam = np.zeros(m)

l = []
v = []
value = f(x)
for i in range(maxIter):
    if i % 1000 == 0:
        print(i, "th iteration, f(x) =", value)
        l.append(beta)
        v.append(value)
    xold = x.copy()
    x = update_x(x, A, b, beta, eta, lam)
    lam += beta * cons(x, A, b)
    if beta*np.max(np.sqrt(eta)*np.linalg.norm(x-xold, 2, axis=1))/np.linalg.norm(b) < eps2:
        rho = rho0
    else:
        rho = 1
    value = f(x)
    if abs(value-f(xold)) < eps1:
        count += 1
        if count == 5:
            print(i, "th iteration, f(x) =", value)
            l.append(beta)
            v.append(value)
            break
    else:
        count = 0
    beta = min(beta*rho, maxBeta)
plt.plot(l)

In [ ]:
plt.plot(v)

In [ ]:
# 调用scipy求值
from scipy.optimize import minimize
def constraint(A, b, m, n):
    def f(x):
        ans = np.zeros(m)
        for i in range(n):
            ans += A[i]@x[i*p:(i+1)*p]
        return ans - b
    return f
con = {'type': 'eq', 'fun': constraint(A, b, m, n)}
def f1(x):
    return np.linalg.norm(x, 1)
res = minimize(f1, x0, constraints=con, tol=1e-10, options={'maxiter': 1e3, 'disp': True})
minValue = res.fun
print("Scipy result:", res.fun)

In [ ]:
eta

In [ ]:
A[0].shape